# Spark - Sales Analytics

## Part 1: Import Data Set and Create DataFrame

### Import Modules and Initiate Session

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType
from pyspark.sql.functions import col, split, trim, to_timestamp, year, month

import os
import zipfile

In [2]:
spark = SparkSession.builder.appName("SparkAnalyticsSales").getOrCreate()

25/03/09 17:24:06 WARN Utils: Your hostname, Cesars-MBP.local resolves to a loopback address: 127.0.0.1; using 192.168.7.230 instead (on interface en0)
25/03/09 17:24:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/03/09 17:24:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Components for DataFrame

In [4]:
schema = StructType([
    StructField("Order ID", StringType(), True),
    StructField("Product", StringType(), True),
    StructField("Quantity Ordered", StringType(), True),
    StructField("Price Each", StringType(), True),
    StructField("Order Date", StringType(), True),
    StructField("Purchase Address", StringType(), True)
])

In [5]:
# Define path and file name
directory_path = "../../data/input"
file_name = "salesdata.zip"

In [6]:
# Create full path for the zip file
zip_file_path = os.path.join(directory_path, file_name)

# Create extraction directory
extraction_directory = os.path.join(directory_path, os.path.splitext(file_name)[0])

# Confirm the extraction directory exists
os.makedirs(extraction_directory, exist_ok=True)

# Open the zip file and extract the contents
with zipfile.ZipFile(zip_file_path, "r") as zip_ref:
    zip_ref.extractall(extraction_directory)

print(f"Extraction complete. Files extracted to: {extraction_directory}")

Extraction complete. Files extracted to: ../../data/input/salesdata


In [7]:
sales_data_df = (spark.read
                .option("header", "true")
                .schema(schema)
                .csv(f"{extraction_directory}/*.csv"))

In [8]:
sales_data_df.show(10)

+--------+--------------------+----------------+----------+--------------+--------------------+
|Order ID|             Product|Quantity Ordered|Price Each|    Order Date|    Purchase Address|
+--------+--------------------+----------------+----------+--------------+--------------------+
|  295665|  Macbook Pro Laptop|               1|      1700|12/30/19 00:01|136 Church St, Ne...|
|  295666|  LG Washing Machine|               1|     600.0|12/29/19 07:03|562 2nd St, New Y...|
|  295667|USB-C Charging Cable|               1|     11.95|12/12/19 18:21|277 Main St, New ...|
|  295668|    27in FHD Monitor|               1|    149.99|12/22/19 15:13|410 6th St, San F...|
|  295669|USB-C Charging Cable|               1|     11.95|12/18/19 12:38|43 Hill St, Atlan...|
|  295670|AA Batteries (4-p...|               1|      3.84|12/31/19 22:58|200 Jefferson St,...|
|  295671|USB-C Charging Cable|               1|     11.95|12/16/19 15:10|928 12th St, Port...|
|  295672|USB-C Charging Cable|         

In [9]:
sales_data_df.printSchema()

root
 |-- Order ID: string (nullable = true)
 |-- Product: string (nullable = true)
 |-- Quantity Ordered: string (nullable = true)
 |-- Price Each: string (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- Purchase Address: string (nullable = true)



In [10]:
sales_data_df.count()

186850

## Part 2: Data Preparation and Cleaning

### Remove All Rows with `NULL` Values and Bad Records

In [ ]:
sales_data_df.filter(col("Order ID").isNull() == True).show(5)

+--------+-------+----------------+----------+----------+----------------+
|Order ID|Product|Quantity Ordered|Price Each|Order Date|Purchase Address|
+--------+-------+----------------+----------+----------+----------------+
|    null|   null|            null|      null|      null|            null|
|    null|   null|            null|      null|      null|            null|
|    null|   null|            null|      null|      null|            null|
|    null|   null|            null|      null|      null|            null|
|    null|   null|            null|      null|      null|            null|
+--------+-------+----------------+----------+----------+----------------+
only showing top 5 rows



In [2]:
sales_data_df.filter(col("Order ID") == "Order ID").show(5)

NameError: name 'sales_data_df' is not defined

In [ ]:
sales_data_df.filter(col("Product") == "Product").show(5)

+--------+-------+----------------+----------+----------+----------------+
|Order ID|Product|Quantity Ordered|Price Each|Order Date|Purchase Address|
+--------+-------+----------------+----------+----------+----------------+
|Order ID|Product|Quantity Ordered|Price Each|Order Date|Purchase Address|
|Order ID|Product|Quantity Ordered|Price Each|Order Date|Purchase Address|
|Order ID|Product|Quantity Ordered|Price Each|Order Date|Purchase Address|
|Order ID|Product|Quantity Ordered|Price Each|Order Date|Purchase Address|
|Order ID|Product|Quantity Ordered|Price Each|Order Date|Purchase Address|
+--------+-------+----------------+----------+----------+----------------+
only showing top 5 rows



In [ ]:
sales_data_df.filter(col("Quantity Ordered") == "Quantity Ordered").show(5)

+--------+-------+----------------+----------+----------+----------------+
|Order ID|Product|Quantity Ordered|Price Each|Order Date|Purchase Address|
+--------+-------+----------------+----------+----------+----------------+
|Order ID|Product|Quantity Ordered|Price Each|Order Date|Purchase Address|
|Order ID|Product|Quantity Ordered|Price Each|Order Date|Purchase Address|
|Order ID|Product|Quantity Ordered|Price Each|Order Date|Purchase Address|
|Order ID|Product|Quantity Ordered|Price Each|Order Date|Purchase Address|
|Order ID|Product|Quantity Ordered|Price Each|Order Date|Purchase Address|
+--------+-------+----------------+----------+----------+----------------+
only showing top 5 rows



In [ ]:
sales_data_df.filter(col("Price Each") == "Price Each").show(5)

+--------+-------+----------------+----------+----------+----------------+
|Order ID|Product|Quantity Ordered|Price Each|Order Date|Purchase Address|
+--------+-------+----------------+----------+----------+----------------+
|Order ID|Product|Quantity Ordered|Price Each|Order Date|Purchase Address|
|Order ID|Product|Quantity Ordered|Price Each|Order Date|Purchase Address|
|Order ID|Product|Quantity Ordered|Price Each|Order Date|Purchase Address|
|Order ID|Product|Quantity Ordered|Price Each|Order Date|Purchase Address|
|Order ID|Product|Quantity Ordered|Price Each|Order Date|Purchase Address|
+--------+-------+----------------+----------+----------+----------------+
only showing top 5 rows



In [ ]:
sales_data_df.filter(col("Order Date") == "Order Date").show(5)

+--------+-------+----------------+----------+----------+----------------+
|Order ID|Product|Quantity Ordered|Price Each|Order Date|Purchase Address|
+--------+-------+----------------+----------+----------+----------------+
|Order ID|Product|Quantity Ordered|Price Each|Order Date|Purchase Address|
|Order ID|Product|Quantity Ordered|Price Each|Order Date|Purchase Address|
|Order ID|Product|Quantity Ordered|Price Each|Order Date|Purchase Address|
|Order ID|Product|Quantity Ordered|Price Each|Order Date|Purchase Address|
|Order ID|Product|Quantity Ordered|Price Each|Order Date|Purchase Address|
+--------+-------+----------------+----------+----------+----------------+
only showing top 5 rows



In [ ]:
sales_data_df.filter(col("Purchase Address") == "Purchase Address").show(5)

+--------+-------+----------------+----------+----------+----------------+
|Order ID|Product|Quantity Ordered|Price Each|Order Date|Purchase Address|
+--------+-------+----------------+----------+----------+----------------+
|Order ID|Product|Quantity Ordered|Price Each|Order Date|Purchase Address|
|Order ID|Product|Quantity Ordered|Price Each|Order Date|Purchase Address|
|Order ID|Product|Quantity Ordered|Price Each|Order Date|Purchase Address|
|Order ID|Product|Quantity Ordered|Price Each|Order Date|Purchase Address|
|Order ID|Product|Quantity Ordered|Price Each|Order Date|Purchase Address|
+--------+-------+----------------+----------+----------+----------------+
only showing top 5 rows



In [ ]:
sales_data_clean_df = sales_data_df.filter(col("Order ID") != "Order ID")
sales_data_clean_df = sales_data_clean_df.filter(col("Product") != "Product")
sales_data_clean_df = sales_data_clean_df.filter(col("Quantity Ordered") != "Quantity Ordered")
sales_data_clean_df = sales_data_clean_df.filter(col("Price Each") != "Price Each")
sales_data_clean_df = sales_data_clean_df.filter(col("Order Date") != "Order Date")
sales_data_clean_df = sales_data_clean_df.filter(col("Purchase Address") != "Purchase Address")

In [ ]:
sales_data_clean_df = sales_data_clean_df.na.drop("any")

In [ ]:
sales_data_clean_df.count()

185950

### Extract City and State from Address into New Columns

In [ ]:
sales_data_clean_df.show(3, truncate=False)

+--------+--------------------+----------------+----------+--------------+--------------------------------------+
|Order ID|Product             |Quantity Ordered|Price Each|Order Date    |Purchase Address                      |
+--------+--------------------+----------------+----------+--------------+--------------------------------------+
|295665  |Macbook Pro Laptop  |1               |1700      |12/30/19 00:01|136 Church St, New York City, NY 10001|
|295666  |LG Washing Machine  |1               |600.0     |12/29/19 07:03|562 2nd St, New York City, NY 10001   |
|295667  |USB-C Charging Cable|1               |11.95     |12/12/19 18:21|277 Main St, New York City, NY 10001  |
+--------+--------------------+----------------+----------+--------------+--------------------------------------+
only showing top 3 rows



In [ ]:
address_elements = split(col("Purchase Address"), ",")

In [ ]:
sales_data_clean_df = (sales_data_clean_df.withColumn("City", trim(address_elements.getItem(1)))
                                         .withColumn("State", split(trim(address_elements.getItem(2)), " ").getItem(0)))

sales_data_clean_df.show(3, truncate=False)

+--------+--------------------+----------------+----------+--------------+--------------------------------------+-------------+-----+
|Order ID|Product             |Quantity Ordered|Price Each|Order Date    |Purchase Address                      |City         |State|
+--------+--------------------+----------------+----------+--------------+--------------------------------------+-------------+-----+
|295665  |Macbook Pro Laptop  |1               |1700      |12/30/19 00:01|136 Church St, New York City, NY 10001|New York City|NY   |
|295666  |LG Washing Machine  |1               |600.0     |12/29/19 07:03|562 2nd St, New York City, NY 10001   |New York City|NY   |
|295667  |USB-C Charging Cable|1               |11.95     |12/12/19 18:21|277 Main St, New York City, NY 10001  |New York City|NY   |
+--------+--------------------+----------------+----------+--------------+--------------------------------------+-------------+-----+
only showing top 3 rows



### Rename and Change Data Types

In [ ]:
sales_data_updated_df = (sales_data_clean_df.withColumnRenamed("Order ID", "OrderID")
                                           .withColumnRenamed("Quantity Ordered", "Quantity")
                                           .withColumnRenamed("Price Each", "Price")
                                           .withColumnRenamed("Order Date", "OrderDate")
                                           .withColumnRenamed("Purchase Address", "StoreAddress"))

sales_data_updated_df.show(3, truncate=False)

+-------+--------------------+--------+-----+--------------+--------------------------------------+-------------+-----+
|OrderID|Product             |Quantity|Price|OrderDate     |StoreAddress                          |City         |State|
+-------+--------------------+--------+-----+--------------+--------------------------------------+-------------+-----+
|295665 |Macbook Pro Laptop  |1       |1700 |12/30/19 00:01|136 Church St, New York City, NY 10001|New York City|NY   |
|295666 |LG Washing Machine  |1       |600.0|12/29/19 07:03|562 2nd St, New York City, NY 10001   |New York City|NY   |
|295667 |USB-C Charging Cable|1       |11.95|12/12/19 18:21|277 Main St, New York City, NY 10001  |New York City|NY   |
+-------+--------------------+--------+-----+--------------+--------------------------------------+-------------+-----+
only showing top 3 rows



In [ ]:
sales_data_updated_df.dtypes

[('OrderID', 'string'),
 ('Product', 'string'),
 ('Quantity', 'string'),
 ('Price', 'string'),
 ('OrderDate', 'string'),
 ('StoreAddress', 'string'),
 ('City', 'string'),
 ('State', 'string')]

In [ ]:
sales_data_updated_df = (sales_data_updated_df.withColumn("OrderID", col("OrderID").cast(IntegerType()))
                                             .withColumn("Quantity", col("Quantity").cast(IntegerType()))
                                             .withColumn("Price", col("Price").cast(FloatType()))
                                             .withColumn("OrderDate", to_timestamp(col("OrderDate"), "MM/dd/yy HH:mm")))

In [ ]:
sales_data_updated_df.show(10, truncate=False)

+-------+--------------------------+--------+------+-------------------+-----------------------------------------+-------------+-----+
|OrderID|Product                   |Quantity|Price |OrderDate          |StoreAddress                             |City         |State|
+-------+--------------------------+--------+------+-------------------+-----------------------------------------+-------------+-----+
|295665 |Macbook Pro Laptop        |1       |1700.0|2019-12-30 00:01:00|136 Church St, New York City, NY 10001   |New York City|NY   |
|295666 |LG Washing Machine        |1       |600.0 |2019-12-29 07:03:00|562 2nd St, New York City, NY 10001      |New York City|NY   |
|295667 |USB-C Charging Cable      |1       |11.95 |2019-12-12 18:21:00|277 Main St, New York City, NY 10001     |New York City|NY   |
|295668 |27in FHD Monitor          |1       |149.99|2019-12-22 15:13:00|410 6th St, San Francisco, CA 94016      |San Francisco|CA   |
|295669 |USB-C Charging Cable      |1       |11.95 |201

In [ ]:
sales_data_updated_df.printSchema()

root
 |-- OrderID: integer (nullable = true)
 |-- Product: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Price: float (nullable = true)
 |-- OrderDate: timestamp (nullable = true)
 |-- StoreAddress: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)



### Add New Columns: ReportYear and ReportMonth

In [ ]:
sales_data_updated_df = (sales_data_updated_df.withColumn("ReportYear", year(col("OrderDate")))
                                             .withColumn("ReportMonth", month(col("OrderDate"))))

In [ ]:
sales_data_updated_df.show(10, truncate=False)

+-------+--------------------------+--------+------+-------------------+-----------------------------------------+-------------+-----+----------+-----------+
|OrderID|Product                   |Quantity|Price |OrderDate          |StoreAddress                             |City         |State|ReportYear|ReportMonth|
+-------+--------------------------+--------+------+-------------------+-----------------------------------------+-------------+-----+----------+-----------+
|295665 |Macbook Pro Laptop        |1       |1700.0|2019-12-30 00:01:00|136 Church St, New York City, NY 10001   |New York City|NY   |2019      |12         |
|295666 |LG Washing Machine        |1       |600.0 |2019-12-29 07:03:00|562 2nd St, New York City, NY 10001      |New York City|NY   |2019      |12         |
|295667 |USB-C Charging Cable      |1       |11.95 |2019-12-12 18:21:00|277 Main St, New York City, NY 10001     |New York City|NY   |2019      |12         |
|295668 |27in FHD Monitor          |1       |149.99|

In [ ]:
sales_data_updated_df.printSchema()

root
 |-- OrderID: integer (nullable = true)
 |-- Product: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Price: float (nullable = true)
 |-- OrderDate: timestamp (nullable = true)
 |-- StoreAddress: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- ReportYear: integer (nullable = true)
 |-- ReportMonth: integer (nullable = true)



### Export DataFrame to Parquet

In [ ]:
export_path = f"../../data/output/sales_analytics"

sales_data_updated_df.write.mode("overwrite").partitionBy("ReportYear", "ReportMonth").parquet(export_path)

## Part 3: Sales Analytics

Prerequisite: Read the Sales Data from the Parquet directory for the 2019 year only.

### Identify the *best* month in terms of sales 

### Identify the city which sold the most products

### Identify the ideal time to display advertisements to maximize the likelihood of customers buying products

### Identify the products often sold together in New York state